In [1]:
%matplotlib inline
from IPython.core.display import display
from itertools import combinations
from tqdm import tqdm_notebook as tqdm
import pickle
import pandas
import numpy

from scipy.stats import skew, kurtosis

In [2]:
with open('final_tree.pkl', 'rb') as f:
    final_tree = pickle.load(f)
use_cols = ['cnum', 'pred', 'new', 'vmax', 'vmin', 'vavg']#, 'vmed', 'vstd', 'vskew', 'vkurt']

In [ ]:
df = pandas.read_csv('../data/test.csv')
submit = pandas.read_csv('submit.csv')
df['pred'] = submit['is_duplicate'].values

df2 = pandas.read_csv('../data/train.csv')
pos_rate = df2['is_duplicate'].sum() / df2.shape[0]
df2['pred'] = df2['is_duplicate'] / pos_rate * 0.165


import networkx as nx
from itertools import combinations
G = nx.Graph()

#edges = [tuple(x) for x in df[['question1', 'question2', 'pred']].values]
#G.add_weighted_edges_from(edges)
map_score = dict(((x[0], x[1]), x[2]) for x in df[['question1', 'question2', 'pred']].values)

#edges = [tuple(x) for x in df2[['question1', 'question2', 'pred']].values]
#G.add_weighted_edges_from(edges)
map_score2 = dict(((x[0], x[1]), x[2]) for x in df2[['question1', 'question2', 'pred']].values)
map_score.update(map_score2)
edges = [(k[0], k[1], v) for k, v in map_score.items()]
G.add_weighted_edges_from(edges)

import copy
cnt = 0
cliques = sorted(list(nx.find_cliques(G)), key=lambda x: (len(x), max(map(str, x))))

In [ ]:
map_result = copy.deepcopy(map_score)
map_cnum = {}
map_data = {}

for cli in tqdm(cliques):
    #if len(cli) < 2:
    #    continue
    if len(cli) == 1:
        q1 = cli[0]
        print(q1)
        map_result[q1, q1] = 1.
        map_cnum[q1, q1] = 1
        map_data[q1, q1] = (1., 1., 1.)
        continue
    keys = {}
    for q1, q2 in combinations(cli, 2):
        if (q1, q2) in map_score:
            keys[q1, q2] = map_score[q1, q2]
        elif (q2, q1) in map_score:
            keys[q2, q1] = map_score[q2, q1]
        elif (q1, q2) in map_score2:
            keys[q1, q2] = map_score2[q1, q2]
        elif (q2, q1) in map_score2:
            keys[q2, q1] = map_score2[q2, q1]
        else:
            raise Exception('no edge {}'.format((q1, q2)))
            
    lval = list(keys.values())
    val_max = numpy.max(lval)
    val_min = numpy.min(lval)
    val_avg = numpy.mean(lval)
    """
    val_med = numpy.median(lval)
    val_std = numpy.std(lval)
    val_skew = skew(lval)
    val_kurt = kurtosis(lval)
    """
    
    if val_avg > 0.4:#avg_pos:
        val = val_max
        keys = {k: numpy.max([val, map_result[k]]) for k in keys}
    elif val_avg > 0.05:
        val = val_avg
        keys = {k: numpy.max([val, map_result[k]]) for k in keys}
    else:
        val = val_min
        keys = {k: numpy.min([val, map_result[k]]) for k in keys}

    map_result.update(keys)
    keys = {k: len(cli) for k in keys}
    map_cnum.update(keys)
    keys = {k: (val_max, val_min, val_avg) for k in keys}
    map_data.update(keys)

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


How do going to gym?
How would you Google Maps?
How you've had sex?
I universe?
What are the best dating in India?
What does the Malayalam word mean?
What is annual revenue?
What is boot?
What is hedge fund have its back office operations in-house or outsourced to a fund admin?
What is in cricket?
What is the difference between middle school and high school?
What is the police code?
What spiritual?
What training?


In [ ]:
list_val = []
list_data = []
for q1, q2 in tqdm(df[['question1', 'question2']].values):
    key = (q1, q2)
    new_pred = map_result[q1, q2]
    if key not in map_cnum:
        map_cnum[q1, q2] = 1
        map_data[q1, q2] = (new_pred, new_pred, new_pred)
    list_val.append(map_result[q1, q2])
    cnum = map_cnum.get((q1, q2), -1)
    data = list(map_data[q1, q2])
    pred = map_score[key]
    
    #'cnum', 'pred', 'new', 'vmax','vmin', 'vavg'
    list_data.append([cnum, pred, new_pred] + data)
    
list_data = pandas.DataFrame(list_data, columns=use_cols)[['cnum', 'pred', 'new', 'vmax', 'vmin', 'vavg']].values
if list_data.shape[1] != final_tree.feature_importances_.shape[0]:                                                                                                                                                                                                                                           
    raise Exception('Not match feature num: %s %s' % (x_test.shape[1], n_features))  
    
pred = final_tree.predict_proba(list_data)[:, 1]
submit['is_duplicate'] = pred#list_val
submit.to_csv('submit_clique.csv', index=False)

In [ ]:
map_dup = dict(((x[0], x[1]), x[2]) for x in df2[['question1', 'question2', 'is_duplicate']].values)
list_res = []
for key, new in map_score2.items():
    new = map_result[key]
    try:
        label = map_dup[key]
    except:
        continue
    pred = map_score2[key]
    new_pred = new
    data = list(map_data.get(key, (0, 0, 0)))
    new_pred = new
    cnum = map_cnum.get(key, 0)
    list_res.append([ label, cnum,pred, new_pred] + data)

aaa = pandas.DataFrame(list_res, columns=['label'] + use_cols)
from tfidf_k import calc_weight
from sklearn.metrics import log_loss, roc_auc_score
sw = calc_weight(aaa['label'].values)
print(roc_auc_score(aaa['label'].values, aaa['pred'].values, sample_weight=sw))
print(log_loss(aaa['label'].values, aaa['pred'].values, sample_weight=sw))
print(roc_auc_score(aaa['label'].values, aaa['new'].values, sample_weight=sw))
print(log_loss(aaa['label'].values, aaa['new'].values, sample_weight=sw))

In [ ]:
bbb = pandas.DataFrame(list_data, columns=['cnum', 'pred', 'new', 'vmax', 'vmin', 'vavg'])

pred = final_tree.predict_proba(bbb)[:, 1]
bbb['new2'] = pred
display(bbb['new2'].hist())

In [ ]:
list_data[0].shap

In [ ]:
bbb

In [ ]:
pred = final_tree.predict_proba(aaa[use_cols])[:, 1]
aaa['new2'] = pred
print(roc_auc_score(aaa['label'].values, aaa['new2'].values, sample_weight=sw))
print(log_loss(aaa['label'].values, aaa['new2'].values, sample_weight=sw))

In [ ]:
print(roc_auc_score(aaa['label'].values, aaa['pred'].values, sample_weight=sw))
print(log_loss(aaa['label'].values, aaa['pred'].values , sample_weight=sw))
print(roc_auc_score(aaa['label'].values, aaa['new'].values, sample_weight=sw))
print(log_loss(aaa['label'].values, aaa['new'].values, sample_weight=sw))

In [ ]:
1.0
0.132888890229
0.99937155985
0.112793364804